### Loading the required libraries

In [1]:
import os
import sys
import cv2
import math
import datetime
import numpy as np
import logging as log
import matplotlib.pyplot as plt
from openvino.inference_engine import IENetwork, IECore, IEPlugin

### Class for working with the Inference Engine

In [2]:
class Network:
    '''
    Load and store information for working with the Inference Engine,
    and any loaded models.
    '''

    def __init__(self):
        self.plugin = None
        self.network = None
        self.input_blob = None
        self.output_blob = None
        self.exec_network = None
        self.infer_request = None

    def load_model(self, model, device="GPU"):
        '''
        Load the model given IR files.
        Defaults to CPU as device for use in the workspace.
        Synchronous requests made within.
        '''
        model_xml = model
        model_bin = os.path.splitext(model_xml)[0] + ".bin"

        # Initialize the plugin
        self.plugin = IECore()

        # Read the IR as a IENetwork
        self.network = IENetwork(model=model_xml, weights=model_bin)

        # Load the IENetwork into the plugin
        self.exec_network = self.plugin.load_network(
            self.network, device_name=device)

        # Get the input layer
        self.input_blob = next(iter(self.network.inputs))
        self.output_blob = next(iter(self.network.outputs))

        return

    def get_input_shape(self):
        '''
        Gets the input shape of the network
        '''
        return self.network.inputs[self.input_blob].shape

    def async_inference(self, image):
        '''
        Makes an asynchronous inference request, given an input image.
        '''
        self.exec_network.start_async(
            request_id=0, inputs={self.input_blob: image})
        return

    def wait(self):
        '''
        Checks the status of the inference request.
        '''
        status = self.exec_network.requests[0].wait(-1)
        return status

    def extract_output(self):
        '''
        Returns a list of the results for the output layer of the network.
        '''
        return self.exec_network.requests[0].outputs[self.output_blob]

### Initialize the Inference Engine

In [3]:
plugin = Network()

### Load the network model into the IE

In [4]:
plugin.load_model("models/ppn-model-2.xml", "HETERO:GPU,CPU")

### Check the input shape

In [5]:
print(plugin.network.batch_size)

40000


In [6]:
net_input_shape = plugin.get_input_shape()
print(net_input_shape)

[40000, 10]


### Creating input

In [7]:
img_size = 200
scale = 0.3
z_size = 7
pattern_change_speed = 0.5

In [8]:
def createInputVec(z, x, y):
    r = math.sqrt(((x * scale - (img_size * scale / 2))**2) + (
        (y * scale - (img_size * scale / 2))**2))
    z_size = len(z)
    input = np.random.rand(1, z_size + 3)

    for i in range(z_size):
        input[0][i] = z[i] * scale

    input[0][z_size] = x * scale
    input[0][z_size + 1] = y * scale
    input[0][z_size + 2] = r
    return input

### Process output after inference

In [9]:
def run_and_plot(fps, seconds):

    frames = fps * seconds
    
    z = np.random.rand(z_size)
    directions = np.ones(z_size)

    for frame in range(frames):

        reverse_directions = np.where(np.logical_or(z > 100, z < -100), -1, 1)
        directions = directions * reverse_directions
        change = np.random.rand(z_size) * pattern_change_speed
        z = z + change * directions

        input_batch = np.zeros((img_size, img_size, z_size + 3))
        for i in range(img_size):

            for j in range(img_size):
                input_batch[i, j] = createInputVec(z, i, j)

        input_batch.resize(img_size * img_size, z_size + 3)

        # Perform inference on the input
        plugin.async_inference(input_batch)

        # Get the output of inference
        if plugin.wait() == 0:
            output_frame = plugin.extract_output()

        output_frame = np.resize(output_frame, (img_size, img_size, 3))
        output_frame = (output_frame * 255).astype(np.uint8)
        
        # displaying each output_frame
        imgplot = plt.imshow(output_frame)
        plt.show()
        
        if (frame + 1) % fps == 1:
            print("\nSec {:03d}:".format(int(frame / fps) + 1), end=" ")
        print("{:3d}".format(frame + 1), end=" ")
    print("\n")